In [ ]:
%pip install --upgrade openai

In [1]:
import openai
import psycopg2
import pandas as pd

# filepath ='/home/azureuser/cloudfiles/code/Users/supakit/SimilaritySearch/Mon/'

In [2]:
openai.api_type = "azure"
openai.api_base = "https://fusion-chatbot-demo.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = "f07d23e10600494f8235496bbf5054b9"

In [55]:
host = "pureelawvec.postgres.database.azure.com"
dbname = "postgres"
user = "pgadmin"
password = "Puree1234"
sslmode = "require"
# Construct connection string

In [56]:
conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)
conn = psycopg2.connect(conn_string) 
print("Connection established")
cursor = conn.cursor()
# Fetch all rows from table

Connection established


In [17]:
import pandas as pd
import os

df = pd.read_csv("criminal-datasets.csv")
df['text'] = df['text'].astype(str) + df['notes'].astype(str)
df['text'] = df['text'].str.replace('nan', '')
df = df.drop(['article','notes'], axis = 1)
df = df.drop(df.index[0:9])
df = df.reset_index()
df = df.drop('index', axis = 1)
df

,text
0,มาตรา ๑ ในประมวลกฎหมายนี้ (๑) “โดยทุจริต” หมา...
1,มาตรา ๒ บุคคลจักต้องรับโทษในทางอาญาต่อเมื่อได...
2,มาตรา ๓ ถ้ากฎหมายที่ใช้ในขณะกระทำความผิดแตกต่...
3,มาตรา ๔ ผู้ใดกระทำความผิดในราชอาณาจักร ต้องรั...
4,มาตรา ๕ ความผิดใดที่การกระทำแม้แต่ส่วนหนึ่งส่...
...,...
439,มาตรา ๓๙๔ ผู้ใดไล่ ต้อน หรือทำให้สัตว์ใด ๆ เข...
440,มาตรา ๓๙๕ ผู้ใดควบคุมสัตว์ใด ๆ ปล่อยปละละเลยใ...
441,มาตรา ๓๙๖ ผู้ใดทิ้งซากสัตว์ซึ่งอาจเน่าเหม็น ใ...
442,มาตรา ๓๙๗ ผู้ใดกระทำด้วยประการใด ๆ ต่อผู้อื่น ...


In [41]:
import requests, uuid, json


# You can pass more than one object in body.
def thai2eng(input):
    key = "041e6b79b96146eabafaf524d78911be"
    endpoint = "https://api.cognitive.microsofttranslator.com"
    location = "southeastasia"
    path = '/translate'
    constructed_url = endpoint + path

    params = {
        'api-version': '3.0',
        'from': 'th',
        'to': ['en']
    }

    headers = {
        'Ocp-Apim-Subscription-Key': key,
        'Ocp-Apim-Subscription-Region': location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }
    # translated = []
    # for string in input:
    body = [{
        'text': input
    }]
    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()
    # translated.append(response[0]["translations"][0]['text'])
    # print(json.dumps(response, sort_keys=True, ensure_ascii=False, indent=4, separators=(',', ': ')))
    # print(response[0]["translations"][0]['text'])
    return response[0]["translations"][0]['text']

def eng2thai(input):
    key = "041e6b79b96146eabafaf524d78911be"
    endpoint = "https://api.cognitive.microsofttranslator.com"
    location = "southeastasia"
    path = '/translate'
    constructed_url = endpoint + path

    params = {
        'api-version': '3.0',
        'from': 'en',
        'to': ['th']
    }

    headers = {
        'Ocp-Apim-Subscription-Key': key,
        'Ocp-Apim-Subscription-Region': location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }
    # translated = []
    # for string in input:
    body = [{
        'text': input
    }]
    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()
    # translated.append(response[0]["translations"][0]['text'])
    # print(json.dumps(response, sort_keys=True, ensure_ascii=False, indent=4, separators=(',', ': ')))
    # print(response[0]["translations"][0]['text'])
    return response[0]["translations"][0]['text']

In [44]:
import pandas as pd
import re

df = pd.read_csv("law.csv")
df['Content'] = df['Content'].str.replace("\n", " ")
# Extract sections using regular expressions
pattern = r'Section \d+\..*?(?=Section \d+\.|$)'
df['Section'] = df['Content'].apply(lambda x: re.findall(pattern, x, re.DOTALL))
# Explode the list of sections into separate rows
df = df.explode('Section')
df_text = df.drop('Content', axis= 1)
# # Reset index
df_text.reset_index(drop=True, inplace=True)
df = df_text.dropna()
# for row in df_text.Content:
#     if row < 20:
#         df_text.drop(row)

In [ ]:
import numpy as np
x = np.squeeze(df.Section.values)
sentence_list = [x for x in x]
sentence_list

In [ ]:
translated = []
for item in sentence_list:
    translated.append(eng2thai(item))
translated

In [ ]:
from sentence_transformers import SentenceTransformer
embed_list = []
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentence_list)
embed_list.append(embeddings.tolist())
print(embeddings)


In [10]:
vectorstore, only_vectorstore, text = [], [], []
embed_model = "text-embedding-ada-002"
Vectorstore, Only_Vectorstore = [], [] # consist of 20 vectors of chunk (sentence1 , sentence2, sentence3, ... ,sentence20)
for idx in range(len(sentence_list)):
  embedding_ = openai.Embedding.create(input=sentence_list[idx],engine=embed_model)
  Vectorstore.append(embedding_)

for item in Vectorstore:
  for data_item in item['data']:
    Vector = data_item['embedding']
    Only_Vectorstore.append(Vector)

In [ ]:
try:
    # cursor.execute("CREATE EXTENSION IF NOT EXISTS pg_trgm;")
    cursor.execute("DROP TABLE Thai_Law")
    cursor.execute("DROP TABLE Eng_Law")
    cursor.execute("CREATE EXTENSION IF NOT EXISTS vector;")
    cursor.execute(""" 
    CREATE TABLE Criminal_Law(
        id INTEGER PRIMARY KEY,
        embedding vector,
        text_eng TEXT,
        text_thai TEXT
    );""")
    for i in range(len(sentence_list)):  
        cursor.execute("INSERT INTO Criminal_Law(id, embedding, text_eng, text_thai) VALUES (%s, %s, %s, %s);",   
                    (i, embed_list[0][i], sentence_list[i], translated[i]))
        print(sentence_list[i]) 
        print(embed_list[0][i])
        conn.commit()  
except Exception as e:
        print(f"Error: {e}")
        conn.rollback()

In [61]:
# def text2vec(input):
#     # input = translator(input)
#     embedding = openai.Embedding.create(input = input, engine = "text-embedding-ada-002")
#     return embedding['data'][0]['embedding']

def text2vec(input):
    embed_list = []
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    embeddings = model.encode(input)
    embed_list.append(embeddings.tolist())
    return embed_list[0]

def search_vectordb(question):
    search_text = []
    try:
         cursor.execute(f"""SELECT text_thai, cosine_distance('{text2vec(question)}', embedding) as cosign
         FROM Criminal_Law
         ORDER BY cosine_distance('{text2vec(question)}', embedding) ASC LIMIT 7
         """)
         query = cursor.fetchall()
         text_eng_results = [row[0] for row in query] 
         return text_eng_results
    except Exception as e:  
            print(f"Error: {e}")
            conn.rollback()
    return "null"

#WHERE cosine_distance('{text2vec(question)}', embedding) < {start}

In [ ]:
search_vectordb(thai2eng("เผาบ้าน"))

In [41]:
conn.commit()
cursor.close()
conn.close()